# G: Explain Models

## imports

In [26]:
#for formatting
import jupyter_black

jupyter_black.load(
    lab=False,
)

In [27]:
import copy
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
import numpy as np
import mlflow
from patsy import dmatrices  # for formula parsing

import json  # for reading signature

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

import h2o

import shap
from datetime import datetime
import os

import importlib
import re
import pickle
import dill

## INPUT 

In [28]:
# Research tracking
runid = "29c8fae222aa4413a11af560c7182b28" # pull from previous file
# mlflow_tracking_uri = "file:D:/Stuff/OneDrive/MLflow"
# mlflow.set_tracking_uri(mlflow_tracking_uri)

forprod = True
if forprod:
    experiment_name = "P1-AnalyzeTrades_f_core"
else:
    experiment_name = "Development"
    
mlflow.set_experiment(experiment_name)
experiment_details = mlflow.get_experiment_by_name(experiment_name)

In [29]:
## pull information

XY_df = pd.read_csv("output/e_resultcleaned.csv")
XY_df["weight"] = 1

In [30]:
## pull information from mlflow and decide model type

# TODO H2O signature missing

mlflow.end_run()
mlflow.start_run(run_id=runid)


def parse_mlflow_info(run_info):
    metrics = run_info.data.metrics
    params = run_info.data.params
    tags = run_info.data.tags
    return metrics, params, tags


metrics, params, tags = parse_mlflow_info(mlflow.get_run(runid))

mlflow.end_run()

formula_clean = params["formula"].replace("\n", "")

model_type = "general"
if "h2o" in str(json.loads(tags["mlflow.log-model.history"])[0]["flavors"]).lower():
    model_type = "h2o"
    print(f"model type is {model_type}")

In [31]:
## pull model from tracking uri

# tracking_uri = mlflow.get_tracking_uri()

artifact_loc = (
    str(experiment_details.artifact_location)
    .replace("file:", "")
    .replace("file:", "")
    .replace("///", "")
)

# try pickle first, otherwise try H2O
if model_type == "h2o":
    # for h2o models
    # h2o.init()
    # mdl = h2o.import_mojo(f'{artifact_loc}/{runid}/artifacts/')
    logged_model = f"runs:/{runid}/model"
    mdl = mlflow.pyfunc.load_model(logged_model)

else:
    mdl = pickle.load(open(f"{artifact_loc}/{runid}/artifacts/model/model.pkl", "rb"))

In [32]:
## parse data

mlflow.end_run()
mlflow.start_run(run_id=runid)

if len(formula_clean) > 1:
    y, X = dmatrices(formula_clean, XY_df, return_type="dataframe")
else:
    X = XY_df.copy()
    y = XY_df[tags["target"]]


# add columns if necessary, can only add, not remove extra cols
cols_required = list(
    pd.DataFrame(
        json.loads(
            json.loads(tags["mlflow.log-model.history"])[0]["signature"]["inputs"]
        )
    )["name"]
)

add_cols = list(set(cols_required) - set(list(X.columns)))
X[add_cols] = 0

# extra columns in dataset
print(
    "extra columns in expanded dataset: "
    + str(list(set(list(X.columns)) - set(cols_required)))
)

X = X[cols_required]  # ensure X is in correct order and complete for model

mlflow.end_run()

extra columns in expanded dataset: ['DATE_YAHOOFINANCE', 'OPEN_DATE', 'CLOSE_^GSPC_200MA', 'OPEN_YEAR', 'OPEN_PRICE', 'UNNAMED:_0', 'STOP', 'EPS1', 'COMMENTS', 'DATE', 'EPS2', 'AAII_BULLISH_BULLISH_AVERAGE', 'AAII_BULLISH_8-WEEK_MOV_AVG', 'QUANTITY', 'AAII_0_LEVEL_0_REPORTED_DATE', 'PCTRETURN', 'ACTION', 'weight', 'AAII_BULLISH_AVERAGE_+ST._DEV.', 'CURRENT_PRICE', 'TIME', 'AAII_1_LEVEL_0_REPORTED_BULLISH', 'PNL', 'TICKER', 'WEIGHT', 'CLOSEACT', 'CLOSE_DATE', 'FILENAME', 'AGE', 'AAII_DATE', 'CONS_SENT_INDEX', 'UNNAMED:_0.2', 'PCT_RET_FINAL', 'FYEPSNXT', 'COMPANY_NAME_(IN_ALPHABETICAL_ORDER)', 'CASH_CHG_(PNL)', 'AAII_S&P_500_WEEKLY_CLOSE', 'COMMENTS.1', 'STARTDATE', 'UNNAMED:_6', 'Unnamed: 0', 'PRICE', 'AAII_2_LEVEL_0_REPORTED_NEUTRAL', 'LASTUPDATED', 'AAII_BULLISH_AVERAGE_-_ST._DEV.', 'AT_PRICE', 'COMM_TOT', 'AAII_3_LEVEL_0_REPORTED_BEARISH', 'FYEND', 'CLOSE_PRICE', 'CONS_SENT_DATE', 'CLOSE_^GSPC', 'AAII_4_LEVEL_0_REPORTED_TOTAL', 'COMMISSION', 'DAYSTOFYEND', 'QTYCHG', 'UNNAMED:_8', 'UN

In [33]:
## train test, repeat from earlier

# X_train, X_test, y_train, y_test, XY_train, XY_test = train_test_split(
#     X, y, XY_df, test_size=0.33, random_state=42)

In [34]:
## predict full dataset

if model_type == "h2o":
    y_pred = mdl.predict(X)
else:
    y_pred = mdl.predict(X)

# h2o.cluster().shutdown(prompt=False)  # if want to end earlier

In [35]:
## class


class H2ORegWrapper:
    def __init__(self, h2o_model, feature_names):
        self.h2o_model = h2o_model
        self.feature_names = feature_names

    def predict(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1, -1)
        # self.dataframe= pd.DataFrame(X, columns=self.feature_names)
        self.predictions = self.h2o_model.predict(X).as_data_frame().values
        return self.predictions.astype("float64")

In [36]:
## lightgbm pipeline


def sub_gbm(X, y_pred):
    """creates explainer based on H2O X & y frames"""
    import lightgbm as lgb
    import shap
    import h2o

    X2_df = X.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))
    y2_pred_df = y_pred

    # TODO create categorical pipeline

    gbm_mdl = lgb.LGBMRegressor(n_jobs=-1)
    gbm_mdl.fit(X2_df, y2_pred_df)

    explainer = shap.Explainer(gbm_mdl.booster_)

    return explainer

## summarize overall results

In [37]:
mlflow.end_run()
mlflow.start_run(run_id=runid)

if "pipeline" in str(type(mdl)).lower():
    model_type = "pipeline"

# Create object that can calculate shap values
if model_type == "h2o":
    # print('h2o explanation')
    # h2o_wrapper = H2ORegWrapper(mdl,X.columns)
    # explainer = shap.SamplingExplainer(h2o_wrapper.predict,h2o.H2OFrame(X[0:100]))

    # TODO use lightgbm with astype('category') to create train model
    explainer = sub_gbm(X, y_pred)
elif model_type == "pipeline":
    explainer = shap.Explainer(mdl[-1])
else:
    # assume sklearn etc.
    explainer = shap.Explainer(mdl)

# save expected value
if len(explainer.expected_value.shape) > 0:
    ev = explainer.expected_value[0]
    explainer.expected_value = ev

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
# TODO get rid of sample below
if model_type == "pipeline":
    shap_values = explainer(mdl[0].transform(X))
else:
    shap_values = explainer(X)  # gets full shap_value descriptions

# ensure slicing by column names work
shap_values.feature_names = list(X.columns)

# Make plot. Index of [1] is explained in text below.
# shap.summary_plot(shap_values, X_train)

f = plt.gcf()

# Make plot to save
# shap.summary_plot(shap_values, X,show=False,) # not as informative as beeswarm

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d%m%Y_%H%M%S")
print("date and time =", dt_string)

if model_type == "pipeline":
    categorical_names = list(X.select_dtypes(include=["object"]).columns)
    col_idx = list(np.where(np.isin(shap_values.feature_names, categorical_names))[0])

    shap_cat = copy.deepcopy(shap_values)
    shap_cat.data = np.array(shap_values.data, dtype="object")
    res_arr = (
        mdl[0]
        .transformers_[1][1][1]
        .inverse_transform(
            pd.DataFrame(shap_cat.data[:, col_idx], columns=[categorical_names])
        )
    )
    for i, loc in enumerate(col_idx):
        shap_cat.data[:, loc] = res_arr[:, i]

shap.plots.beeswarm(shap_values, show=False)
plt.tight_layout()
plt.savefig(f"summary_plot_{dt_string}.png", bbox_inches="tight")
plt.show()

mlflow.log_metric("expected_val", ev)
mlflow.log_artifact(f"summary_plot_{dt_string}.png")

os.remove(f"summary_plot_{dt_string}.png")

mlflow.end_run()

date and time = 19082023_134304


No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [38]:
## save explainer

mlflow.end_run()
mlflow.start_run(run_id=runid)

with open(f"explainer.pkl", "wb") as handle:
    dill.dump(explainer, handle, recurse=True)

mlflow.log_artifact(f"explainer.pkl")

os.remove(f"explainer.pkl")

mlflow.end_run()

In [39]:
## check highest

top_trades = XY_df[XY_df["PCT_RET_FINAL"] > 1]
top_trades.head()

,Unnamed: 0,%_TO_STOP,%_TO_TARGET,AAII_0_LEVEL_0_REPORTED_DATE,AAII_1_LEVEL_0_REPORTED_BULLISH,AAII_2_LEVEL_0_REPORTED_NEUTRAL,AAII_3_LEVEL_0_REPORTED_BEARISH,AAII_4_LEVEL_0_REPORTED_TOTAL,AAII_BULLISH_8-WEEK_MOV_AVG,AAII_BULLISH_AVERAGE_+ST._DEV.,AAII_BULLISH_AVERAGE_-_ST._DEV.,AAII_BULLISH_BULL-BEAR_SPREAD,AAII_BULLISH_BULLISH_AVERAGE,AAII_DATE,AAII_S&P_500_WEEKLY_CLOSE,AAII_S&P_500_WEEKLY_HIGH,AAII_S&P_500_WEEKLY_LOW,ACTION,AGE,AT_PRICE,CASH_CHG_(PNL),CATEGORY,CLOSEACT,CLOSE_DATE,CLOSE_PRICE,CLOSE_^GSPC,CLOSE_^GSPC_200MA,CLOSE_^VIX,COMMENTS,COMMENTS.1,COMMISSION,COMM_TOT,COMPANY_NAME_(IN_ALPHABETICAL_ORDER),CONS_SENT_DATE,CONS_SENT_INDEX,CURRENT_PRICE,DATE,DATE_YAHOOFINANCE,DAYOFWEEK0MON,DAYSTOFYEND,EPS1,EPS2,FILENAME,FYEND,FYEPSNXT,GROWTH_0.5TO0.75,LASTUPDATED,OPENACT,OPEN_DATE,OPEN_PRICE,OPEN_YEAR,PCTRETURN,PCT_RET_FINAL,PNL,PRICE,QTYCHG,QUANTITY,ROIC_(BW_ROA_ROE),SP500FROM200MA,STARTDATE,STOP,SYMBOL,TARGET,TGT_FWD_P_E,TICKER,TIME,UNNAMED:_0,UNNAMED:_0.1,UNNAMED:_0.2,UNNAMED:_6,UNNAMED:_8,WEIGHT,YEARS_TO_NORMALIZATION,weight
47,47,-0.1473,0.0045,2015-06-25 00:00:00,0.355556,0.427778,0.216667,1.000001,0.267944,0.478443,0.276547,0.138889,0.377495,2015-06-25,2108.580000,2129.870000,2082.100000,BOT,1,_NA_,"($7,781.40)",Value,S,2021-06-17 12:40:33,1184.7050,2063.110107,2053.895403,18.23,_NA_,_NA_,1.00,-1.030805,_NA_,2015-06-30,96.1,_NA_,2015-05-14,2015-06-30,3,_NA_,_NA_,_NA_,PCM-Tracking - LogHist.csv,_NA_,_NA_,0.1,_NA_,B,2015-06-30 00:00:00,579.659697,2015,_NA_,1.043447,3024.22,778.040,10,5,0.15,0.004486,_NA_,_NA_,MKL,_NA_,24.84,_NA_,9:37:15 AM,1914,15,47,USD,_NA_,0.80,6,1
607,607,-0.1473,0.0045,2016-04-07 00:00:00,0.321918,0.463470,0.214612,1.000000,0.314051,0.478443,0.276547,0.107306,0.377495,2016-04-07,2066.659912,2075.070068,2042.560059,SLD,2,_NA_,"$5,937.77",LGrowth,S,2018-09-10 15:53:07,1937.0100,2061.719971,2013.898400,14.85,_NA_,_NA_,1.13,-2.379502,_NA_,2016-03-31,91.0,_NA_,2016-04-12,2016-04-12,1,_NA_,_NA_,_NA_,PCM-Tracking - LogHist.csv,_NA_,_NA_,0.1,_NA_,B,2016-04-12 13:04:47,600.140000,2016,_NA_,2.227137,6682.97,593.890,10,5,0.15,0.023746,_NA_,_NA_,AMZN,_NA_,24.84,_NA_,10:14:42,958,607,607,USD,U1060261,0.64,6,1
699,699,-0.1473,0.0045,2016-05-12 00:00:00,0.204082,0.482993,0.312925,1.000000,0.280639,0.478443,0.276547,-0.108843,0.377495,2016-05-12,2064.459961,2084.870117,2039.449951,SLD,2,_NA_,"$36,126.81",Value,S,2018-10-03 14:56:30,232.3027,2064.110107,2012.624700,14.41,_NA_,_NA_,2.39,-2.878473,_NA_,2016-04-30,89.0,_NA_,2016-05-12,2016-05-12,3,_NA_,_NA_,_NA_,PCM-Tracking - LogHist.csv,_NA_,_NA_,0.1,_NA_,B,2016-05-12 11:47:57,90.209450,2016,_NA_,1.574633,3551.17,90.323,400,25,0.15,0.025581,_NA_,_NA_,AAPL,_NA_,24.84,_NA_,10:45:03,979,705,699,USD,U1060261,0.64,6,1
740,740,-0.1473,0.0045,2016-08-04 00:00:00,0.297872,0.434397,0.267730,0.999999,0.300769,0.478443,0.276547,0.030142,0.377495,2016-08-04,2163.790000,2178.290000,2147.580000,BOT,2,_NA_,"($4,837.00)",LGrowth,S,2018-07-13 10:13:51,398.2735,2182.870117,2044.974099,11.39,_NA_,_NA_,1.00,-2.052966,_NA_,2016-07-31,90.0,_NA_,2016-08-05,2016-08-05,4,_NA_,_NA_,_NA_,PCM-Tracking - LogHist.csv,_NA_,_NA_,0.1,_NA_,B,2016-08-05 10:54:18,96.720000,2016,_NA_,3.116677,3014.45,96.720,50,10,0.15,0.067432,_NA_,_NA_,NFLX,_NA_,24.84,_NA_,Aug 5 10:54:18,927,740,740,USD,U1060261,0.64,6,1
741,741,-0.1473,0.0045,2016-08-04 00:00:00,0.297872,0.434397,0.267730,0.999999,0.300769,0.478443,0.276547,0.030142,0.377495,2016-08-04,2163.790000,2178.290000,2147.580000,BOT,2,_NA_,"($4,837.00)",LGrowth,S,2020-04-21 11:00:08,428.8900,2182.870117,2044.974099,11.39,_NA_,_NA_,1.00,-2.191949,_NA_,2016-07-31,90.0,_NA_,2016-08-05,2016-08-05,4,_NA_,_NA_,_NA_,PCM-Tracking - LogHist.csv,_NA_,_NA_,0.1,_NA_,B,2016-08-05 10:54:18,96.720000,2016,_NA_,3.433526,4981.36,96.720,50,15,0.15,0.067432,_NA_,_NA_,NFLX,_NA_,24.84,_NA_,Aug 5 10:54:18,1434,741,741,USD,U1060261,0.64,6,1


## plot partial dependence for vars

In [40]:
# make plots

mlflow.end_run()
mlflow.start_run(run_id=runid)

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
print("date and time =", dt_string)

for var in cols_required:
    fig, ax = plt.subplots()

    shap_slice = shap_values[:, var]

    try:
        shap.plots.scatter(shap_slice, ax=ax, show=False, color=shap_values)
    except:
        shap.plots.scatter(shap_slice, ax=ax, show=False)

    if var in categorical_names:
        # get integer labels
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

        orig_list = ax.get_xticks()
        new_list = np.insert(
            mdl[0].transformers_[1][1][1].categories_[categorical_names.index(var)],
            0,
            "Unknown",
        )

        for i in range(len(orig_list) - len(new_list)):
            try:
                new_list = np.append(new_list, orig_list[i + len(new_list)])
            except:
                pass

        ax.set_xticks(orig_list)
        ax.set_xticklabels(new_list)

    else:
        # deprecated
        # shap.dependence_plot(var, shap_values.values, X, ax=ax, show=False)

        int_labels, col_bins = pd.cut(X[var], bins=10, retbins=True, labels=False)

        shap_col_df = pd.DataFrame(
            {
                var: col_bins[int_labels],
                "expected_value_shap": shap_values.values[:, X.columns.get_loc(var)],
            }
        )

        shap_col_grp_df = shap_col_df.groupby(var).mean()

        shap_col_grp_df.plot(ax=ax)

    # TODO add
    # shap.plots.partial_dependence(var, mdl.predict, X,  model_expected_value=True)
    # f = plt.gcf()

    plt.tight_layout()
    plt.savefig(f"shappdp_{var}_{dt_string}.png", bbox_inches="tight")
    plt.show()

    mlflow.log_artifact(f"shappdp_{var}_{dt_string}.png")

    os.remove(f"shappdp_{var}_{dt_string}.png")

mlflow.end_run()

date and time = 20230819_134304


Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [41]:
## test waterfallplot

main_file = importlib.import_module("P1-AnalyzeTrades_h_predictresult")

res = main_file.predict_return(
    mlflow_tracking_uri="mlruns",  # assumes development
    experiment_name=experiment_name,
    run_id=runid,
    inputs=X.loc[[0], :],
    explain=True,
    show_plot=True,
    preloaded_model=mdl,
)
res

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


(   predicted_ret
 0       0.027095,
 .values =
 array([[ 4.34764709e-03,  3.17495607e-03, -8.04926568e-03,
          1.11680762e-03,  1.15648232e-18,  2.79622792e-03,
         -6.99944515e-04,  1.15648232e-18, -8.95989056e-03,
         -1.32710434e-02,  2.55131478e-02, -5.21693183e-02]])
 
 .base_values =
 array([0.0732961])
 
 .data =
 array([[ 2.48400000e+01,  6.00000000e+00,  1.82299995e+01,
          1.38889000e-01, -1.47300000e-01,  4.50000000e-03,
          1.00000000e-01,  1.50000000e-01,  1.00000000e+00,
          4.48645271e-03,  0.00000000e+00,  4.00000000e+00]]),
    TGT_FWD_P_E  YEARS_TO_NORMALIZATION  CLOSE_^VIX  \
 0     0.004348                0.003175   -0.008049   
 
    AAII_BULLISH_BULL-BEAR_SPREAD     %_TO_STOP  %_TO_TARGET  GROWTH_0.5TO0.75  \
 0                       0.001117  1.156482e-18     0.002796           -0.0007   
 
    ROIC_(BW_ROA_ROE)  DAYOFWEEK0MON  SP500FROM200MA   OPENACT  CATEGORY  
 0       1.156482e-18       -0.00896       -0.013271  0.025513 -0

In [42]:
res

(   predicted_ret
 0       0.027095,
 .values =
 array([[ 4.34764709e-03,  3.17495607e-03, -8.04926568e-03,
          1.11680762e-03,  1.15648232e-18,  2.79622792e-03,
         -6.99944515e-04,  1.15648232e-18, -8.95989056e-03,
         -1.32710434e-02,  2.55131478e-02, -5.21693183e-02]])
 
 .base_values =
 array([0.0732961])
 
 .data =
 array([[ 2.48400000e+01,  6.00000000e+00,  1.82299995e+01,
          1.38889000e-01, -1.47300000e-01,  4.50000000e-03,
          1.00000000e-01,  1.50000000e-01,  1.00000000e+00,
          4.48645271e-03,  0.00000000e+00,  4.00000000e+00]]),
    TGT_FWD_P_E  YEARS_TO_NORMALIZATION  CLOSE_^VIX  \
 0     0.004348                0.003175   -0.008049   
 
    AAII_BULLISH_BULL-BEAR_SPREAD     %_TO_STOP  %_TO_TARGET  GROWTH_0.5TO0.75  \
 0                       0.001117  1.156482e-18     0.002796           -0.0007   
 
    ROIC_(BW_ROA_ROE)  DAYOFWEEK0MON  SP500FROM200MA   OPENACT  CATEGORY  
 0       1.156482e-18       -0.00896       -0.013271  0.025513 -0

In [43]:
res[1]

.values =
array([[ 4.34764709e-03,  3.17495607e-03, -8.04926568e-03,
         1.11680762e-03,  1.15648232e-18,  2.79622792e-03,
        -6.99944515e-04,  1.15648232e-18, -8.95989056e-03,
        -1.32710434e-02,  2.55131478e-02, -5.21693183e-02]])

.base_values =
array([0.0732961])

.data =
array([[ 2.48400000e+01,  6.00000000e+00,  1.82299995e+01,
         1.38889000e-01, -1.47300000e-01,  4.50000000e-03,
         1.00000000e-01,  1.50000000e-01,  1.00000000e+00,
         4.48645271e-03,  0.00000000e+00,  4.00000000e+00]])

In [44]:
## end mlflow and h2o

mlflow.end_run()
try:
    h2o.cluster().shutdown(prompt=False)  # if want to end earlier
except:
    pass

In [45]:
## backup functions

# https://scikit-learn.org/stable/auto_examples/linear_model/plot_tweedie_regression_insurance_claims.html
# def gini(actual, pred, sample_weight = None):
#     #ignores weights
#     assert (len(actual) == len(pred))
#     allv = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
#     allv = allv[np.lexsort((allv[:, 2], -1 * allv[:, 1]))]
#     totalLosses = allv[:, 0].sum()
#     giniSum = allv[:, 0].cumsum().sum() / totalLosses

#     giniSum -= (len(actual) + 1) / 2.
#     return giniSum / len(actual)

# def gini_normalized(truth, predictions, sample_weight=None):
#     return gini(truth, predictions) / gini(truth, truth)

# https://stackoverflow.com/questions/48999542/more-efficient-weighted-gini-coefficient-in-python
# def gini(x, sample_weight=None):
#     # The rest of the code requires numpy arrays.
#     x = np.asarray(x)
#     if sample_weight is not None:
#         w = np.asarray(sample_weight)
#         sorted_indices = np.argsort(x)
#         sorted_x = x[sorted_indices]
#         sorted_w = w[sorted_indices]
#         # Force float dtype to avoid overflows
#         cumw = np.cumsum(sorted_w, dtype=float)
#         cumxw = np.cumsum(sorted_x * sorted_w, dtype=float)
#         return (np.sum(cumxw[1:] * cumw[:-1] - cumxw[:-1] * cumw[1:]) /
#                 (cumxw[-1] * cumw[-1]))
#     else:
#         sorted_x = np.sort(x)
#         n = len(x)
#         cumx = np.cumsum(sorted_x, dtype=float)
#         # The above formula, with all weights equal to 1 simplifies to:
#         return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

# def gini_normalized(y_actual, y_pred, sample_weight=None):
#     """
#     Gini coefficient based on two lists and optional weight list, all of same shape
#     """
#     ans = (gini(y_pred , sample_weight=sample_weight)
#            / gini(y_actual , sample_weight=sample_weight)
#     )
#     return ans